Question 15.2

In the videos, we saw the “diet problem”. (The diet problem is one of the first large-scale optimization
problems to be studied in practice. Back in the 1930’s and 40’s, the Army wanted to meet the nutritional
requirements of its soldiers while minimizing the cost.) In this homework you get to solve a diet problem with real data. The data is given in the file diet.xls. 

1.	Formulate an optimization model (a linear program) to find the cheapest diet that satisfies the maximum and minimum daily nutrition constraints, and solve it using PuLP.  Turn in your code and the solution. (The optimal solution should be a diet of air-popped popcorn, poached eggs, oranges, raw iceberg lettuce, raw celery, and frozen broccoli. UGH!)
2.	Please add to your model the following constraints (which might require adding more variables) and solve the new model:
	- a.	If a food is selected, then a minimum of 1/10 serving must be chosen. (Hint: now you will need two variables for each food i: whether it is chosen, and how much is part of the diet. You’ll also need to write a constraint to link them.)
	- b.	Many people dislike celery and frozen broccoli. So at most one, but not both, can be selected.
	- c.	To get day-to-day variety in protein, at least 3 kinds of meat/poultry/fish/eggs must be selected. [If something is ambiguous (e.g., should bean-and-bacon soup be considered meat?), just call it whatever you think is appropriate – I want you to learn how to write this type of constraint, but I don’t really care whether we agree on how to classify foods!]

If you want to see what a more full-sized problem would look like, try solving your models for the file diet_large.xls, which is a low-cholesterol diet model (rather than minimizing cost, the goal is to minimize cholesterol intake).  I don’t know anyone who’d want to eat this diet – the optimal solution includes dried chrysanthemum garland, raw beluga whale flipper, freeze-dried parsley, etc. – which shows why it’s necessary to add additional constraints beyond the basic ones we saw in the video!
	[Note: there are many optimal solutions, all with zero cholesterol, so you might get a different one.  It probably won’t be much more appetizing than mine.]


In [270]:
#!pip install PuLP
#!pip install xlrd
import pandas as pd
import pulp
from pulp import *

## EDA

In [271]:
excel_file = r"C:\Users\Clair\OneDrive\Documents\GitHub\omsa\ISYE 6501\Homework 11\diet.xls"

# read by default 1st sheet of an excel file
diet_data = pd.read_excel(excel_file)

# table preview
display(diet_data)

,Foods,Price/ Serving,Serving Size,Calories,Cholesterol mg,Total_Fat g,Sodium mg,Carbohydrates g,Dietary_Fiber g,Protein g,Vit_A IU,Vit_C IU,Calcium mg,Iron mg
0,Frozen Broccoli,0.16,10 Oz Pkg,73.8,0.0,0.8,68.2,13.6,8.5,8.0,5867.4,160.2,159.0,2.3
1,"Carrots,Raw",0.07,1/2 Cup Shredded,23.7,0.0,0.1,19.2,5.6,1.6,0.6,15471.0,5.1,14.9,0.3
2,"Celery, Raw",0.04,1 Stalk,6.4,0.0,0.1,34.8,1.5,0.7,0.3,53.6,2.8,16.0,0.2
3,Frozen Corn,0.18,1/2 Cup,72.2,0.0,0.6,2.5,17.1,2.0,2.5,106.6,5.2,3.3,0.3
4,"Lettuce,Iceberg,Raw",0.02,1 Leaf,2.6,0.0,0.0,1.8,0.4,0.3,0.2,66.0,0.8,3.8,0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,"Crm Mshrm Soup,W/Mlk",0.65,1 C (8 Fl Oz),203.4,19.8,13.6,1076.3,15.0,0.5,6.1,153.8,2.2,178.6,0.6
63,"Beanbacn Soup,W/Watr",0.67,1 C (8 Fl Oz),172.0,2.5,5.9,951.3,22.8,8.6,7.9,888.0,1.5,81.0,2.0
64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65,NaN,NaN,Minimum daily intake,1500.0,30.0,20.0,800.0,130.0,125.0,60.0,1000.0,400.0,700.0,10.0


In [272]:
diet_data.describe()

,Price/ Serving,Calories,Cholesterol mg,Total_Fat g,Sodium mg,Carbohydrates g,Dietary_Fiber g,Protein g,Vit_A IU,Vit_C IU,Calcium mg,Iron mg
count,64.000000,66.000000,66.000000,66.000000,66.000000,66.000000,66.000000,66.000000,66.000000,66.000000,66.000000,66.000000
mean,0.327188,190.918182,21.615152,6.392424,364.486364,24.727273,7.139394,8.312121,920.250000,92.968182,82.106061,2.296970
std,0.254536,354.515790,49.660770,12.547158,528.629735,57.824484,34.016991,14.797760,2422.290914,615.827721,208.060208,5.406434
min,0.020000,2.600000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.900000,0.000000
25%,0.145000,73.975000,0.000000,0.500000,17.175000,5.000000,0.000000,2.025000,16.050000,0.000000,6.900000,0.325000
50%,0.270000,110.650000,0.000000,2.900000,144.850000,15.250000,0.800000,4.200000,100.200000,1.900000,21.450000,0.700000
75%,0.460000,168.950000,19.425000,7.075000,389.025000,22.450000,2.000000,8.175000,699.775000,9.175000,58.025000,2.175000
max,0.990000,2500.000000,240.000000,72.200000,2000.000000,450.000000,250.000000,100.000000,15471.000000,5000.000000,1500.000000,40.000000


In [273]:
# column names
diet_columns = list(diet_data.columns)
diet_columns

['Foods',
 'Price/ Serving',
 'Serving Size',
 'Calories',
 'Cholesterol mg',
 'Total_Fat g',
 'Sodium mg',
 'Carbohydrates g',
 'Dietary_Fiber g',
 'Protein g',
 'Vit_A IU',
 'Vit_C IU',
 'Calcium mg',
 'Iron mg']

In [274]:
# I see NaNs in the table preview 
diet_data.isnull().values.any()

True

In [275]:
# how many null rows?
diet_data.isnull().sum()

Foods              3
Price/ Serving     3
Serving Size       1
Calories           1
Cholesterol mg     1
Total_Fat g        1
Sodium mg          1
Carbohydrates g    1
Dietary_Fiber g    1
Protein g          1
Vit_A IU           1
Vit_C IU           1
Calcium mg         1
Iron mg            1
dtype: int64

In [276]:
# what are those rows, more importantly i want to know the indices
diet_data[diet_data.isnull().any(axis=1)]

,Foods,Price/ Serving,Serving Size,Calories,Cholesterol mg,Total_Fat g,Sodium mg,Carbohydrates g,Dietary_Fiber g,Protein g,Vit_A IU,Vit_C IU,Calcium mg,Iron mg
64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65,NaN,NaN,Minimum daily intake,1500.0,30.0,20.0,800.0,130.0,125.0,60.0,1000.0,400.0,700.0,10.0
66,NaN,NaN,Maximum daily intake,2500.0,240.0,70.0,2000.0,450.0,250.0,100.0,10000.0,5000.0,1500.0,40.0


In [277]:
# limit rows to just legit food rows
diet_data_clean = diet_data[0:64]
# making sure the null value columns are truly eliminated
diet_data_clean.tail()

,Foods,Price/ Serving,Serving Size,Calories,Cholesterol mg,Total_Fat g,Sodium mg,Carbohydrates g,Dietary_Fiber g,Protein g,Vit_A IU,Vit_C IU,Calcium mg,Iron mg
59,Neweng Clamchwd,0.75,1 C (8 Fl Oz),175.7,10.0,5.0,1864.9,21.8,1.5,10.9,20.1,4.8,82.8,2.8
60,Tomato Soup,0.39,1 C (8 Fl Oz),170.7,0.0,3.8,1744.4,33.2,1.0,4.1,1393.0,133.0,27.6,3.5
61,"New E Clamchwd,W/Mlk",0.99,1 C (8 Fl Oz),163.7,22.3,6.6,992.0,16.6,1.5,9.5,163.7,3.5,186.0,1.5
62,"Crm Mshrm Soup,W/Mlk",0.65,1 C (8 Fl Oz),203.4,19.8,13.6,1076.3,15.0,0.5,6.1,153.8,2.2,178.6,0.6
63,"Beanbacn Soup,W/Watr",0.67,1 C (8 Fl Oz),172.0,2.5,5.9,951.3,22.8,8.6,7.9,888.0,1.5,81.0,2.0


Min and max serving

a.	If a food is selected, then a minimum of 1/10 serving must be chosen. (Hint: now you will need two variables for each food i: whether it is chosen, and how much is part of the diet. You’ll also need to write a constraint to link them.)

In [278]:
# with my eyeballs, i can see Lettuce,Iceberg,Raw has the lowest calorie of 2.6 and Pork has the highest calorie of 710.8, need to confirm with code

# lowest and highest calories
min_calories_index = diet_data_clean['Calories'].idxmin()
max_calories_index = diet_data_clean['Calories'].idxmax()

# what row are these foods in 
print("Index of minimum Calories:", min_calories_index) 
print("Index of maximum Calories:", max_calories_index)

# lets see the data for the foods im anticipating
min_calories_row = diet_data_clean.loc[min_calories_index] # should be Lettuce,Iceberg,Raw
max_calories_row = diet_data_clean.loc[max_calories_index] # should be Pork

print("Row with minimum Calories:\n", min_calories_row)
print("Row with maximum Calories:\n", max_calories_row)


Index of minimum Calories: 4
Index of maximum Calories: 49
Row with minimum Calories:
 Foods              Lettuce,Iceberg,Raw
Price/ Serving                    0.02
Serving Size                    1 Leaf
Calories                           2.6
Cholesterol mg                     0.0
Total_Fat g                        0.0
Sodium mg                          1.8
Carbohydrates g                    0.4
Dietary_Fiber g                    0.3
Protein g                          0.2
Vit_A IU                          66.0
Vit_C IU                           0.8
Calcium mg                         3.8
Iron mg                            0.1
Name: 4, dtype: object
Row with maximum Calories:
 Foods               Pork
Price/ Serving      0.81
Serving Size        4 Oz
Calories           710.8
Cholesterol mg     105.1
Total_Fat g         72.2
Sodium mg           38.4
Carbohydrates g      0.0
Dietary_Fiber g      0.0
Protein g           13.8
Vit_A IU            14.7
Vit_C IU             0.0
Calcium mg      

## 1. Orgnaize data

In [279]:
# selecting food rows (the rows that are not null)
food_data = diet_data_clean
nutrient_constraints = food_data.iloc[:, 3:].reset_index(drop=True) # every row of the diet data clean, 4th column and on
display(food_data.head(2)) # every row of the diet data clean --> diet_data_clean = diet_data[0:64]
display(nutrient_constraints.head(2)) # 4th column and on starting with calories

,Foods,Price/ Serving,Serving Size,Calories,Cholesterol mg,Total_Fat g,Sodium mg,Carbohydrates g,Dietary_Fiber g,Protein g,Vit_A IU,Vit_C IU,Calcium mg,Iron mg
0,Frozen Broccoli,0.16,10 Oz Pkg,73.8,0.0,0.8,68.2,13.6,8.5,8.0,5867.4,160.2,159.0,2.3
1,"Carrots,Raw",0.07,1/2 Cup Shredded,23.7,0.0,0.1,19.2,5.6,1.6,0.6,15471.0,5.1,14.9,0.3


,Calories,Cholesterol mg,Total_Fat g,Sodium mg,Carbohydrates g,Dietary_Fiber g,Protein g,Vit_A IU,Vit_C IU,Calcium mg,Iron mg
0,73.8,0.0,0.8,68.2,13.6,8.5,8.0,5867.4,160.2,159.0,2.3
1,23.7,0.0,0.1,19.2,5.6,1.6,0.6,15471.0,5.1,14.9,0.3


In [280]:
food_data.columns

Index(['Foods', 'Price/ Serving', 'Serving Size', 'Calories', 'Cholesterol mg',
       'Total_Fat g', 'Sodium mg', 'Carbohydrates g', 'Dietary_Fiber g',
       'Protein g', 'Vit_A IU', 'Vit_C IU', 'Calcium mg', 'Iron mg'],
      dtype='object')

In [281]:
# extract costs and nutrient values
food_costs = food_data['Price/ Serving'].tolist()
calories = food_data['Calories'].tolist()
cholestrols = food_data['Cholesterol mg'].tolist()
fats = food_data['Total_Fat g'].tolist()
sodium = food_data['Sodium mg'].tolist()
carbohydrates = food_data['Carbohydrates g'].tolist()
fibers = food_data['Dietary_Fiber g'].tolist()
vitamin_a = food_data['Vit_A IU'].tolist()
vitamin_c = food_data['Vit_C IU'].tolist()
calcium = food_data['Calcium mg'].tolist()
iron = food_data['Iron mg'].tolist()

print(food_costs)

[0.16, 0.07, 0.04, 0.18, 0.02, 0.53, 0.06, 0.31, 0.84, 0.78, 0.27, 0.24, 0.15, 0.32, 0.49, 0.15, 0.16, 0.05, 0.06, 0.09, 0.16, 0.03, 0.05, 0.25, 0.16, 0.23, 0.13, 0.08, 0.11, 0.15, 0.27, 0.33, 0.15, 0.31, 0.28, 0.28, 0.34, 0.32, 0.38, 0.82, 0.52, 0.44, 0.59, 0.83, 0.31, 0.39, 0.08, 0.17, 0.07, 0.81, 0.45, 0.69, 0.04, 0.22, 0.12, 0.19, 0.39, 0.67, 0.71, 0.75, 0.39, 0.99, 0.65, 0.67]


## 2. Pulp object

In [282]:
# create a pulp- object
import pulp

prob = pulp.LpProblem("diet_optimization", sense=pulp.LpMinimize)
prob

diet_optimization:
MINIMIZE
None
VARIABLES

## Decision variables